In [2]:
#Installieren der benötigten Bibliotheken

#import pyzed.sl as sl
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import pandas as pd


### Datengewinnung aus der SVO-Datei

In [ ]:
# Funktion für Datenverarbeitung aus der SVO-Datei (Muss nicht unbedingt ausgeführt werden, da es einige Zeit in anspruch nimmt, die resultierenden CSV-Dateien sind bereits vorhanden)

def setup_zed_camera(svo_path):
    """Richtet die ZED-Kamera ein und gibt das Kamerainstanz-Objekt zurück."""
    zed = sl.Camera()

    # Input-Typ basierend auf der SVO-Datei
    input_type = sl.InputType()
    input_type.set_from_svo_file(svo_path)

    # Initialisierungsparameter
    init_params = sl.InitParameters(input_t=input_type, svo_real_time_mode=False)
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA
    init_params.coordinate_units = sl.UNIT.METER
    init_params.depth_maximum_distance = 50  

    # Öffne die Kamera
    if zed.open(init_params) != sl.ERROR_CODE.SUCCESS:
        raise RuntimeError("Kamera konnte nicht geöffnet werden.")

    return zed

def setup_object_detection(zed):
    """Aktiviert die Objektdetektion mit den definierten Parametern."""
    detection_params = sl.ObjectDetectionParameters()
    detection_params.enable_tracking = True
    detection_params.enable_segmentation = True
    detection_params.detection_model = sl.OBJECT_DETECTION_MODEL.MULTI_CLASS_BOX_MEDIUM
    zed.enable_object_detection(detection_params)

    return sl.ObjectDetectionRuntimeParameters(detection_confidence_threshold=40, 
                                               object_class_filter=[sl.OBJECT_CLASS.PERSON])

def setup_positional_tracking(zed):
    """Aktiviert die Positional Tracking-Funktion."""
    tracking_params = sl.PositionalTrackingParameters()
    zed.enable_positional_tracking(tracking_params)

def create_dataframe_from_svo(svo_path):
    """Erstellt einen DataFrame mit den Daten aus der SVO-Datei."""
    zed = setup_zed_camera(svo_path)
    setup_positional_tracking(zed)
    detection_params_rt = setup_object_detection(zed)

    # Daten sammeln
    frame_number = 0
    objects = sl.Objects()
    data = []

    while zed.grab() == sl.ERROR_CODE.SUCCESS:
        zed.retrieve_objects(objects, detection_params_rt)
        for obj in objects.object_list:
            pos = obj.position
            vel = obj.velocity
            data.append({
                "Frame": frame_number,
                "Object ID": obj.id,
                "Position X": pos[0],
                "Position Y": pos[1],
                "Position Z": pos[2],
                "Velocity X": vel[0],
                "Velocity Y": vel[1],
                "Velocity Z": vel[2]
            })
        frame_number += 1

    zed.close()

    # DataFrame erstellen
    df = pd.DataFrame(data)
    return df

# SVO-Dateipfad (Muss angepasst werden, da die Videos nicht auf Git sind)
svo_path = "Aufnahme_Kalibrierung.svo2"
svo_path_1 = "Aufnahme_3_Games.svo"
Kalibrierung = create_dataframe_from_svo(svo_path)
#Kalibrierung.to_csv("Kalibrierung.csv", index=False)
Aufnahme_3_Games = create_dataframe_from_svo(svo_path_1)
#Aufnahme_3_Games.to_csv("Match_Daten_ungereinigt.csv", index=False)


### Extrahierung der Kalibrierungspunkte

In [19]:
Kalibrierung = pd.read_csv("Kalibrierung.csv")

# In diesen Frames befinden sich die Kalibrierungspunkte. (Dies wurde von Hand ermittelt, durch das Anschauen der SVO-Datei)
frames = [156, 964, 664]

# Filtere die Zeilen mit den spezifischen Frame-Werten und Object.ID == 0
selected_rows = Kalibrierung[(Kalibrierung['Frame'].isin(frames)) & (Kalibrierung['Object ID'] == 0)]

# Sortiere die Zeilen entsprechend der Reihenfolge der Frames
selected_rows = selected_rows.set_index('Frame').loc[frames]

# Extrahiere die Werte aus den Spalten 'Position.X', 'Position.Y', 'Position.Z'
src_points = selected_rows[['Position X', 'Position Y', 'Position Z']].to_numpy()


### Manuelle Umwandlung der Object-Id in 0 und 1

In [3]:
# Umwandeln der Objekt-ID in 0 und 1. Wie man sieht hat es ziemlich viele Object Id welche von Zed erkannt wurden.
Match = pd.read_csv("Match_Daten_ungereinigt.csv")
Match = Match[~Match['Object ID'].isin([15,16,20,25,28,29,30,32,33,34,35,37,41,42,43,44,45,47,49,48,50,52,53,54,55,58,60,61,62,63,67,68,69])]
Match['Object ID'] = Match['Object ID'].replace({3: 0, 6: 0, 11: 0, 26: 0})
Match['Object ID'] = Match['Object ID'].replace({2: 1, 4: 1, 5: 1, 7: 1, 8: 1, 12: 1, 14: 1, 17: 1, 18: 1, 19: 1,
    21: 1, 22: 1, 24: 1, 27: 1, 28: 1, 31: 1, 39: 1, 40: 1, 46: 1,
    57: 1, 59: 1, 64: 1, 66: 1})

# Einzelne anpassung einer Objekt-ID
Match = Match[~((Match['Frame'] < 5877) & (Match['Object ID'] == 36))]
Match.loc[(Match['Frame'] >= 5877) & (Match['Object ID'] == 36), 'Object ID'] = 1

# Umwadeln der Spaltennamen
Match.columns = Match.columns.str.strip()
Match = Match.rename(columns={
    'Object ID': 'Object.ID',
    'Position X': 'Position.X',
    'Position Y': 'Position.Y',
    'Position Z': 'Position.Z',
    'Velocity X': 'Velocity.X',
    'Velocity Y': 'Velocity.Y',
    'Velocity Z': 'Velocity.Z'
})


### Transformation der 3D-Koordinaten in 2D-Koordinaten

In [39]:
# Funktion zur Berechnung der Transformationsmatrix
def calculate_transformation_matrix(src_points, dst_points):
    src_points_h = np.hstack((src_points, np.ones((src_points.shape[0], 1))))
    transform_matrix, _, _, _ = np.linalg.lstsq(src_points_h, dst_points, rcond=None)
    return transform_matrix

# Funktion zum Transformieren eines Punktes
def transform_point(P, matrix):
    P_hom = np.append(P, 1)
    return np.dot(P_hom, matrix)

#Eckpunkte des Tenisfeldes welche für die Transformation benötigt werden
dst_points = np.array([
    [0, -11.885],    # P1 2D (unten Mitte der Grundlinie)
    [-4.115, 0],     # P2 2D (links Mitte der Seitenlinie)
    [4.115, 0]       # P3 2D (rechts Mitte der Seitenlinie)
])

# Berechnung der Transformationsmatrix
transformation_matrix = calculate_transformation_matrix(src_points, dst_points)

# Funktion für die Anwendung der Transformation auf das ganze DataFrame
def apply_transformation(row):
    point_3d = np.array([row['Position.X'], row['Position.Y'], row['Position.Z']])
    transformed_point = transform_point(point_3d, transformation_matrix)
    return pd.Series({'Transformed.X': transformed_point[0], 'Transformed.Y': transformed_point[1]})

# Transformation auf das gesamte DataFrame anwenden
Match[['Transformed.X', 'Transformed.Y']] = Match.apply(apply_transformation, axis=1)

# Unnötige Spalten entfernen, einschlislich der Geschwindigkeiten da diese nicht Transformiert wurde. Die geschwindigkeit wird weiter unten neu berechnet
Match = Match.drop(['Position.X', 'Position.Y', 'Position.Z', 'Velocity.X', 'Velocity.Y', 'Velocity.Z'], axis=1)

# Überprüfen ob es noch Object-Id gibt welche nicht 0 oder 1 sind
Match.drop(Match[~Match['Object.ID'].isin([0, 1])].index, inplace=True)

# Entfernung aller Zeilen welche die gleichen Frame und Object-ID haben. Dies sind nur wenige Zeilen welche bei der Manuellen Anpassung der Object-Id entstanden sind.
Match = Match.drop_duplicates(subset=['Frame', 'Object.ID'], keep='first')



### Hinzufügen der Spalten 'Spiel läuft' und 'Game'

In [40]:
# In diesen Intervallen der Frames ist das Tennis-Spiel aktiv
intervals = [
    (347, 666),
    (927, 1232),
    (1443, 1734),
    (2082, 2303),
    (2607, 2866),
    (3450, 3590),
    (3753, 4000),
    (4150, 4350),
    (4672, 5370),
    (6035, 6230),
    (6860, 7325),
    (8146, 8554),
    (9080, 9419)
]

# Funktion, um zu prüfen, ob ein Frame in einem der Intervalle liegt
def is_game_running(frame):
    for start, end in intervals:
        if start <= frame <= end:
            return 'Ja'
    return 'Nein'

# Neue Spalte 'Spiel läuft' erstellen und die Funktion anwenden
Match['Spiel läuft'] = Match['Frame'].apply(is_game_running)

# In diesen Intervallen befindet sich das Spiel in Game 1, 2 oder 3
conditions = [
    (Match['Frame'] >= 347) & (Match['Frame'] <= 2890) & (Match['Spiel läuft'] == 'Ja'),   # Game 1
    (Match['Frame'] >= 4816) & (Match['Frame'] <= 9419) & (Match['Spiel läuft'] == 'Ja'),   # Game 3
    (Match['Frame'] >= 3450) & (Match['Frame'] <= 4815) & (Match['Spiel läuft'] == 'Ja')    # Game 2
]
choices = ['1', '3', '2']

# Neue Spalte 'Game' hinzufügen. Wenn die Frames ausserhalb der Intervalle sind, wird 'Spiel läuft nicht' zurückgegeben
Match['Game'] = np.select(conditions, choices, default='Spiel läuft nicht')



### Hinzufügen der Spalte 'Speed' und Glättung der Geschwindigkeit

In [41]:
# Sortieren nach Object.ID und Frame
Match = Match.sort_values(by=['Object.ID', 'Frame'])

# Berechnung der Differenzen
Match['delta_frame'] = Match['Frame'].diff(periods=3)
Match['delta_x'] = Match['Transformed.X'].diff(periods=3)
Match['delta_y'] = Match['Transformed.Y'].diff(periods=3)

# Berechnung der Distanz und Zeitdifferenz
Match['distance'] = np.sqrt(Match['delta_x']**2 + Match['delta_y']**2)
Match['time_diff'] = Match['delta_frame'] / 30  # 30 FPS

# Berechnung der Geschwindigkeit
Match['Speed'] = np.where(
    Match['delta_frame'] == 3, 
    Match['distance'] / Match['time_diff'], 
    np.nan
)

# Unnötige Spalten entfernen
Match.drop(columns=['delta_frame', 'delta_x', 'delta_y', 'distance', 'time_diff'], inplace=True)

# Zurücksortieren nach Frame
Match = Match.sort_values(by='Frame').reset_index(drop=True)


# Funktion des Savgol Filter welche für die Glättung der Geschwindigkeit gebraucht wird
def apply_savgol_filter(data, col_name, object_id, window_length=5, polyorder=2):
    """
    Wendet den Savitzky-Golay-Filter für eine bestimmte Object.ID an und speichert die geglätteten Werte in der 'Speed'-Spalte.
    
    Parameters:
        data (DataFrame): Der DataFrame, der die zu glättende Spalte enthält.
        col_name (str): Der Name der Spalte, die geglättet werden soll.
        object_id (int): Die Object.ID, für die die Glättung durchgeführt werden soll.
        window_length (int): Die Fenstergröße für den Filter (muss ungerade sein).
        polyorder (int): Der Grad des Polynoms, das für die Glättung verwendet wird.
    
    Returns:
        DataFrame: Der DataFrame mit der geglätteten Spalte für die angegebene Object.ID.
    """
    # Filter nur für die Zeilen mit der angegebenen Object.ID anwenden
    group = data[data['Object.ID'] == object_id]
    
    
    smoothed = savgol_filter(group[col_name], window_length=window_length, polyorder=polyorder)
    
    # Die geglätteten Werte in der 'smoothed'-Spalte speichern
    data.loc[data['Object.ID'] == object_id, 'Speed'] = smoothed
    
    return data

# Anwendung der Funktion für Object.ID = 1 und Object.ID = 0
Match = apply_savgol_filter(Match, col_name='Speed', object_id=1, window_length=5, polyorder=1)
Match = apply_savgol_filter(Match, col_name='Speed', object_id=0, window_length=5, polyorder=1)

# Negative Geschwindigkeiten auf 0 setzen
Match['Speed'] = Match['Speed'].clip(lower=0)

# Geschwindigkeiten oberhalb der Grenze entfernen
Match.loc[Match['Speed'] >= 10, 'Speed'] = np.nan

# Koordinaten entfernen welche offensichtich falsch sind
Match.loc[Match['Transformed.X'] > 10, 'Transformed.X'] = np.nan
Match.loc[Match['Transformed.Y'] < -20, 'Transformed.Y'] = np.nan

# In diesen Frames läuft das Spiel aber es fehlen Koordinaten und Geschwindigkeiten
Match = Match[~Match['Frame'].between(4796, 4804)]
Match = Match.reset_index(drop=True)

### Hinzufügen von Zeilen für Spieler 1, für Frames welche Spieler 1 von Zed nicht erkannt wurde. Dies wird gebraucht um die Geschwindigkeiten miteinander zu vergleichen

In [43]:
# Erstelle eine leere Liste, um die Zeilen für die fehlenden Object.IDs zu speichern
additional_rows = []

# Durchlaufe alle einzigartigen Frames
for frame in Match['Frame'].unique():
    # Finde alle Object.IDs, die für diesen Frame existieren
    object_ids_in_frame = Match[Match['Frame'] == frame]['Object.ID'].unique()
    
    # Wenn nur eine Object.ID für den Frame existiert
    if len(object_ids_in_frame) == 1:
        existing_object_id = object_ids_in_frame[0]
        missing_object_id = 1 if existing_object_id == 0 else 0  # Falls 0, setze auf 1, und umgekehrt
        
        # Hole die Werte der existierenden Zeile
        existing_row = Match[(Match['Frame'] == frame) & (Match['Object.ID'] == existing_object_id)].iloc[0]
        
        # Erstelle eine neue Zeile mit der fehlenden Object.ID und übernehme die Werte für 'Spiel läuft' und 'Game'
        new_row = {
            'Frame': frame,
            'Object.ID': missing_object_id,
            'Transformed.X': np.nan,
            'Transformed.Y': np.nan,
            'Spiel läuft': existing_row['Spiel läuft'],
            'Game': existing_row['Game'],
            'Speed': np.nan
        }
        
        # Füge die neue Zeile der Liste hinzu
        additional_rows.append(new_row)

# Füge die neuen Zeilen dem DataFrame hinzu
Match = pd.concat([Match, pd.DataFrame(additional_rows)], ignore_index=True)
Match = Match.sort_values(by=['Frame']).reset_index(drop=True)




### Spalte 'Spieler schlägt' hinzufügen

In [24]:
# Bedingungen für "Spieler schlägt" für Object.ID == 0. Dies wurde von Hand ermittelt, durch das Anschauen der SVO-Datei
conditions_player0 = (Match['Object.ID'] == 0) & (Match['Frame'].isin([400, 530, 950, 1088, 1456, 1579, 1705, 2112, 2220, 2613, 
                                                                2750, 3865, 4240, 4750, 4960, 5080, 5200, 6055, 6876, 
                                                                7000, 7110, 7200, 8157, 8291, 8455, 9101, 9237]))

# Bedingungen für "Spieler schlägt" für Object.ID == 1. Dies wurde von Hand ermittelt, durch das Anschauen der SVO-Datei
conditions_player1 = (Match['Object.ID'] == 1) & (Match['Frame'].isin([468, 602, 1020, 1134, 1520, 1643, 2170, 2671, 2815, 
                                                                3475, 3802, 3925, 4170, 4278, 4687, 5030, 5147, 5240, 
                                                                6130, 6945, 7050, 7155, 7254, 8205, 8402, 8495, 9157, 9305]))

# Alle anderen Zeilen (wo keine der Bedingungen zutrifft)
conditions_other = ~(conditions_player0 | conditions_player1)

# Neue Spalte 'Spieler schlägt' hinzufügen
Match['Spieler schlägt'] = np.select([conditions_player0, conditions_player1, conditions_other], ['Ja', 'Ja', 'Nein'], default='Nein')

In [25]:
#CSV Datei erstellen, welche im Dashboard verwendet werden kann
#Match.to_csv('Match_Daten.csv', index=False)